In [ ]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import statsmodels.api as sm
import shapely
from tqdm.contrib.concurrent import thread_map, process_map
from glob import glob
pd.set_option('display.max_columns', None)

In [2]:
shorelines = gpd.read_file("Data for testing/NZCCDv1.shp")
shorelines

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry
0,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,1.265403,2004-01-02,0.0,"LINESTRING Z (1728907.5 5916213.248 0, 1728868..."
1,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.307010,2004-01-02,1.0,"LINESTRING Z (1729067.838 5914779.733 0, 17290..."
2,Auckland,KarekareBethells,MW,2000,tod (landward),RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.255243,2004-01-02,2.0,"LINESTRING Z (1729647.881 5911983.994 0, 17296..."
3,Auckland,KarekareBethells,MW,2000,tod,RL,3,1,40000,5.03,1.416425,5.314890,01/02/2004,1.107251,2004-01-02,3.0,"MULTILINESTRING Z ((1728131.079 5917218.888 0,..."
4,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.770375,2004-01-02,4.0,"MULTILINESTRING Z ((1729643.228 5912906.682 0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,West Coast,WoodpeckerBay,MD,1000,EdgeVeg,RL,2,1,18800,4.00,0.630693,3.736680,11/18/1948,1.191368,1948-11-18,19661.0,"LINESTRING Z (1467237.671 5348414.193 0, 14672..."
19662,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.650952,2018-04-04,19662.0,"LINESTRING Z (1444386.944 5008247.367 0, 14443..."
19663,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.179434,2018-04-04,19663.0,"LINESTRING Z (1444860.629 5008686.925 0, 14448..."
19664,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.174825,2018-04-04,19664.0,"LINESTRING Z (1444996.286 5008822.252 0, 14450..."


In [3]:
CPS_error_lookup = {1: 0.43, 2: 0.73, 3: 0.97, 4: 2.07, 5: 8.59}
def calc_UNCY(row):
  # Calculate Total_UNCY
  Ep = row.Pixel_Er
  Ed = CPS_error_lookup[row.CPS]
  Eg = row.Georef_ER
  return np.sqrt(Ep**2 + Eg**2 + Ed**2)
shorelines["new_Total_UNCY"] = shorelines.apply(calc_UNCY, axis=1)
shorelines["UNCY_diff"] = shorelines.Total_UNCY - shorelines.new_Total_UNCY
shorelines[shorelines.UNCY_diff.abs() > 1].sort_values("UNCY_diff")

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry,new_Total_UNCY,UNCY_diff
4567,Gisborne,TeAraroa,AG,2000,"Landfill, edge of vege",RL,5,1,10800,4.0,0.486351,3.769554,05/19/1939,0.492869,1939-05-19,4567.0,"LINESTRING Z (2074482.21 5821331.308 0, 207447...",9.488131,-5.718577
8672,Northland,Mimiwhangata,ER,1000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,0.206609,1942-04-30,8672.0,"LINESTRING Z (1729907.188 6075213.925 0, 17299...",9.479111,-5.309885
8671,Northland,Mimiwhangata,ER,1000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,1.031080,1942-04-30,8671.0,"LINESTRING Z (1729601.032 6075467.108 0, 17295...",9.479111,-5.309885
8670,Northland,Mimiwhangata,ER,1500,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,1.517729,1942-04-30,8670.0,"LINESTRING Z (1729477.877 6076546.628 0, 17294...",9.479111,-5.309885
8669,Northland,Mimiwhangata,ER,2000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,0.190708,1942-04-30,8669.0,"LINESTRING Z (1729897.752 6075258.124 0, 17298...",9.479111,-5.309885
6005,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.179704,2016-01-07,6005.0,"LINESTRING Z (1697813.385 5382563.368 0, 16978...",0.990404,1.080954
6004,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.106993,2016-01-07,6004.0,"LINESTRING Z (1697849.263 5382468.276 0, 16978...",0.990404,1.080954
6002,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.045874,2016-01-07,6002.0,"LINESTRING Z (1698434.258 5381486.247 0, 16984...",0.990404,1.080954
6003,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,1.002103,2016-01-07,6003.0,"LINESTRING Z (1698359.169 5381604.516 0, 16983...",0.990404,1.080954
6219,Marlborough,WairauRiver,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.096524,2016-01-07,6219.0,"LINESTRING Z (1694971.212 5399242.128 0, 16949...",0.990404,1.080954


In [4]:
transects = pd.concat(gpd.read_file(f).set_index("Unique_ID") for f in glob("Data for testing/Unique_ID_transects/*.shp"))
#transects = gpd.read_file("https://uoa-eresearch.github.io/CoastSat/transects_extended.geojson")
#transects = transects[transects.site_id.str.startswith("nzd")].to_crs(2193).drop_duplicates("id").set_index("id")
transects

/home/ubuntu/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/ubuntu/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/ubuntu/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/ubuntu/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/ubuntu/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported.

,geometry
Unique_ID,
2.030544e+11,"LINESTRING Z (1197707.561 4775935.423 0, 11979..."
2.030544e+11,"LINESTRING Z (1197716.835 4775939.162 0, 11979..."
2.030544e+11,"LINESTRING Z (1197726.11 4775942.902 0, 119796..."
2.030544e+11,"LINESTRING Z (1197735.384 4775946.642 0, 11979..."
2.030544e+11,"LINESTRING Z (1197744.659 4775950.381 0, 11979..."
...,...
2.020008e+11,"LINESTRING Z (1616477.032 5429332.43 0, 161634..."
2.020008e+11,"LINESTRING Z (1616467.288 5429330.182 0, 16163..."
2.020008e+11,"LINESTRING Z (1616457.544 5429327.935 0, 16163..."


In [5]:
DSAS_rates = gpd.read_file("Data for testing/kx-nzccd-coastal-change-rates-SHP.zip")
DSAS_rates

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
0,13.0,-0.27,0.80,0.07,3.20,-0.27,0.05,-19.30,24.84,-0.23,0.65,0.09,1.37,4015.900000,Waikato,1.005336e+11,1950-04-01,2022-02-15,71,POINT (1740823.984 5718945.283)
1,9.0,0.58,0.91,0.13,5.14,0.56,0.07,39.42,39.42,0.55,0.93,0.10,1.14,44129.150000,Tasman,2.000050e+11,1950-10-11,2021-03-13,70,POINT (1573804.234 5506243.949)
2,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602827.257 5444493.261)
3,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602821.392 5444501.389)
4,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602815.442 5444509.464)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228608,5.0,0.01,0.05,0.08,1.81,-0.02,0.07,-1.50,3.73,0.00,0.00,0.13,0.97,3223.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784449.725 5926935.288)
228609,5.0,0.06,0.63,0.07,1.45,0.03,0.07,1.91,4.73,0.05,0.36,0.09,0.71,3233.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784439.931 5926939.809)
228610,5.0,0.06,0.64,0.06,1.27,0.03,0.07,1.74,4.23,0.07,0.46,0.10,0.74,3243.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784431.172 5926944.695)
228611,5.0,0.03,0.34,0.06,1.26,0.00,0.07,0.18,2.73,0.04,0.22,0.09,0.70,3253.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784423.016 5926950.491)


In [7]:
DSAS_rates[DSAS_rates.UniqueID == DSAS_rates.UniqueID.iloc[-1]]

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
228612,5.0,0.04,0.67,0.04,0.87,0.03,0.07,1.54,3.06,0.03,0.18,0.08,0.59,3263.05,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784415.426 5926957.131)


In [ ]:
def process_row(Unique_ID):
    transect = transects.geometry[Unique_ID]
    # This function reproduces the statistics described in https://pubs.usgs.gov/of/2021/1091/ofr20211091.pdf
    try:
        shorelines["intersect_point"] = shorelines.intersection(transect)
    except:
        print(Unique_ID)
        raise
    df = shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")
    if len(df) < 3:
        #print(f"{Unique_ID} has only {len(df)} shorelines")
        return {}
    df.Date = pd.to_datetime(df.Date)
    df["YearsSinceBase"] = (df.Date - df.Date.min()).dt.days / 365.25
    transect_origin = shapely.get_point(transect, -1)
    df["Distance"] = df.intersect_point.distance(transect_origin)
    LR = sm.OLS(df.Distance, sm.add_constant(df.YearsSinceBase)).fit()
    low, high = LR.conf_int(alpha=.1).loc["YearsSinceBase"]
    LCI = (high - low) / 2
    WLR = sm.WLS(df.Distance, sm.add_constant(df.YearsSinceBase), weights=1 / df.Total_UNCY ** 2).fit()
    low, high = WLR.conf_int(alpha=.1).loc["YearsSinceBase"]
    WCI = (high - low) / 2
    duration = (df.Date.max() - df.Date.min()).days / 365.25
    assert duration > 0, f"{Unique_ID} has a duration of {duration}"
    NSM = -(df.Distance.iloc[0] - df.Distance.iloc[-1])
    return {
        "UniqueID": Unique_ID,
        "Region": df.Region.value_counts().idxmax(),
        "Start_date": str(df.Date.min().date()),
        "End_date": str(df.Date.max().date()),
        "Duration": round(duration),
        "ShrCount": len(df),
        "NSM": round(NSM, 2),
        "SCE": round(df.intersect_point.apply(lambda p: df.intersect_point.distance(p).max()).max(), 2),
        "EPR": round(NSM / duration, 2),
        "EPRunc": round(np.sqrt(df.Total_UNCY.iloc[0] ** 2 + df.Total_UNCY.iloc[-1] ** 2) / duration, 2),
        "LRR": LR.params.YearsSinceBase.round(2),
        "LRI": LR.params.const.round(2),
        "LCI": round(LCI, 2),
        "LSE": np.sqrt(LR.mse_resid).round(2),
        "LR2": LR.rsquared.round(2),
        "WLR": WLR.params.YearsSinceBase.round(2),
        "WLI": WLR.params.const.round(2),
        "WCI": round(WCI, 2),
        "WSE": np.sqrt(WLR.mse_resid).round(2),
        "WR2": WLR.rsquared.round(2),
        "geometry": transect,
        "Dates": df.Date.dt.strftime("%Y-%m-%d").tolist(),
        "Distances": df.Distance.round(2).tolist(),
    }

display(pd.DataFrame([process_row(100533643879), DSAS_rates.iloc[0]]))
import platform
if platform.system() == "Windows":
    results = thread_map(process_row, transects.index, chunksize=100)
else:
    results = process_map(process_row, transects.index, chunksize=100)
rates = gpd.GeoDataFrame(results, crs=2193)
rates

,UniqueID,Region,Start_date,End_date,Duration,ShrCount,NSM,SCE,EPR,EPRunc,LRR,LRI,LCI,LSE,LR2,WLR,WLI,WCI,WSE,WR2,geometry,Dates,Distances,TCD
0,1.005336e+11,Waikato,1950-04-01,2022-02-15,72,13.0,-19.3,24.85,-0.27,0.05,-0.27,155.79,0.07,3.2,0.8,-0.23,153.97,0.09,1.37,0.65,LINESTRING Z (1740712.4318284073 5718976.23720...,"[1950-04-01, 1974-09-12, 1979-04-01, 1986-11-2...","[157.05, 151.78, 144.95, 145.67, 138.57, 135.4...",NaN
1,1.005336e+11,Waikato,1950-04-01 00:00:00,2022-02-15 00:00:00,71,13.0,-19.3,24.84,-0.27,0.05,-0.27,NaN,0.07,3.2,0.8,-0.23,NaN,0.09,1.37,0.65,POINT (1740823.984043876 5718945.283080289),NaN,NaN,4015.9


  0%|          | 0/228613 [00:00<?, ?it/s]

,UniqueID,Region,Start_date,End_date,Duration,ShrCount,NSM,SCE,EPR,EPRunc,LRR,LRI,LCI,LSE,LR2,WLR,WLI,WCI,WSE,WR2,geometry,Dates,Distances
0,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,28.13,28.13,0.57,0.12,0.59,360.02,1.04,5.75,0.93,0.66,357.98,0.90,1.41,0.96,"LINESTRING Z (1197707.561 4775935.423 0, 11979...","[1958-02-10, 1978-02-25, 2007-02-05]","[362.78, 367.2, 390.91]"
1,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,29.60,29.60,0.60,0.12,0.62,358.33,1.07,5.92,0.93,0.69,356.23,0.93,1.45,0.96,"LINESTRING Z (1197716.835 4775939.162 0, 11979...","[1958-02-10, 1978-02-25, 2007-02-05]","[361.17, 365.98, 390.77]"
2,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,30.68,30.68,0.63,0.12,0.64,357.54,0.85,4.70,0.96,0.70,355.87,0.74,1.15,0.97,"LINESTRING Z (1197726.11 4775942.902 0, 119796...","[1958-02-10, 1978-02-25, 2007-02-05]","[359.79, 366.56, 390.47]"
3,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,30.80,30.80,0.63,0.12,0.64,357.30,0.71,3.90,0.97,0.69,355.92,0.61,0.96,0.98,"LINESTRING Z (1197735.384 4775946.642 0, 11979...","[1958-02-10, 1978-02-25, 2007-02-05]","[359.18, 366.97, 389.98]"
4,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,31.66,31.66,0.65,0.12,0.66,356.25,0.59,3.27,0.98,0.70,355.09,0.51,0.80,0.99,"LINESTRING Z (1197744.659 4775950.381 0, 11979...","[1958-02-10, 1978-02-25, 2007-02-05]","[357.82, 366.74, 389.48]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228608,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,7.0,-256.06,280.29,-4.97,0.10,-1.70,153.41,3.44,97.44,0.17,-1.98,170.29,3.08,23.83,0.25,"LINESTRING Z (1616477.032 5429332.43 0, 161634...","[1969-09-22, 1969-09-22, 1981-01-11, 2010-02-1...","[314.21, 33.91, 68.63, 135.44, 78.3, 51.05, 58..."
228609,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,8.0,-272.22,306.31,-5.29,0.10,-1.58,145.42,3.53,104.92,0.11,-2.28,180.71,3.58,28.75,0.20,"LINESTRING Z (1616467.288 5429330.182 0, 16163...","[1969-09-22, 1969-09-22, 1981-01-11, 2006-12-1...","[316.12, 9.81, 46.87, 178.64, 116.03, 52.22, 3..."
228610,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,7.0,-288.24,55.67,-5.60,0.10,-3.75,218.78,3.61,86.99,0.47,-4.64,265.15,3.25,20.37,0.62,"LINESTRING Z (1616457.544 5429327.935 0, 16163...","[1969-09-22, 1981-01-11, 2006-12-14, 2010-02-1...","[318.95, 24.48, 135.29, 95.8, 31.5, 16.5, 30.71]"
228611,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,8.0,-274.46,302.58,-5.33,0.10,-1.72,148.57,3.38,100.55,0.14,-2.30,178.44,3.25,26.12,0.24,"LINESTRING Z (1616435.945 5429323.553 0, 16163...","[1969-09-22, 1969-09-22, 1981-01-11, 2006-12-1...","[319.74, 17.16, 48.95, 156.51, 110.25, 53.65, ..."


In [24]:
(rates.set_index("UniqueID").select_dtypes(include=['float64']) - DSAS_rates.set_index("UniqueID").select_dtypes(include=['float64'])).describe()

,Duration,EPR,EPRunc,LCI,LR2,LRI,LRR,LSE,NSM,SCE,ShrCount,TCD,WCI,WLI,WLR,WR2,WSE
count,0.0,228538.000000,228538.000000,228538.00000,228538.000000,0.0,228538.000000,228538.000000,228538.000000,228538.000000,228538.000000,0.0,228538.000000,0.0,228538.000000,228538.000000,228538.000000
mean,NaN,0.006615,-0.000016,0.02089,-0.003757,NaN,0.007616,1.079170,0.556067,2.111388,0.079129,NaN,0.031936,NaN,0.000106,-0.005454,0.484322
std,NaN,0.295152,0.007607,0.39098,0.066892,NaN,0.231718,10.798423,20.640159,25.478328,0.936745,NaN,0.470617,NaN,0.309712,0.076218,5.276215
min,NaN,-10.040000,-0.290000,-24.69000,-1.000000,NaN,-8.390000,-93.800000,-573.710000,-197.230000,-5.000000,NaN,-24.470000,NaN,-11.090000,-0.980000,-53.240000
25%,NaN,0.000000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
50%,NaN,0.000000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
75%,NaN,0.000000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
max,NaN,10.650000,0.530000,29.10000,0.980000,NaN,15.510000,288.010000,456.810000,734.250000,33.000000,NaN,47.890000,NaN,11.250000,0.990000,179.400000


In [28]:
points = pd.concat(gpd.read_file(f) for f in glob("Data for testing/Unique_ID_points/*.shp"))
points

,Unique_ID,geometry
0,2.030564e+11,POINT Z (1197669.751 4785165.138 0)
1,2.030564e+11,POINT Z (1197679.006 4785168.973 0)
2,2.030564e+11,POINT Z (1197857.086 4785235.367 0)
3,2.030564e+11,POINT Z (1197866.606 4785238.437 0)
4,2.030564e+11,POINT Z (1197876.127 4785241.506 0)
...,...,...
139133,1.004750e+11,POINT Z (1758086.752 5426055.275 0)
139134,1.004750e+11,POINT Z (1758091.623 5426064.015 0)
139135,1.004750e+11,POINT Z (1758096.813 5426072.563 0)
139136,1.004750e+11,POINT Z (1758102.158 5426081.017 0)


In [29]:
rates.drop(columns="geometry").to_parquet("rates_with_timeseries.parquet")


In [30]:
rates_with_timeseries = gpd.GeoDataFrame(rates.drop(columns="geometry").merge(points, left_on="UniqueID", right_on="Unique_ID").drop(columns="Unique_ID"))
rates_with_timeseries["lat"] = rates_with_timeseries.geometry.to_crs(4326).y
rates_with_timeseries["lon"] = rates_with_timeseries.geometry.to_crs(4326).x
rates_with_timeseries.drop(columns="geometry").to_parquet("rates_with_timeseries.parquet")
rates_with_timeseries

,UniqueID,Region,Start_date,End_date,Duration,ShrCount,NSM,SCE,EPR,EPRunc,LRR,LRI,LCI,LSE,LR2,WLR,WLI,WCI,WSE,WR2,Dates,Distances,geometry,lat,lon
0,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,28.13,28.13,0.57,0.12,0.59,360.02,1.04,5.75,0.93,0.66,357.98,0.90,1.41,0.96,"[1958-02-10, 1978-02-25, 2007-02-05]","[362.78, 367.2, 390.91]",POINT Z (1197795.608 4775717.066 0),-47.049447,167.704192
1,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,29.60,29.60,0.60,0.12,0.62,358.33,1.07,5.92,0.93,0.69,356.23,0.93,1.45,0.96,"[1958-02-10, 1978-02-25, 2007-02-05]","[361.17, 365.98, 390.77]",POINT Z (1197806.859 4775715.903 0),-47.049464,167.704338
2,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,30.68,30.68,0.63,0.12,0.64,357.54,0.85,4.70,0.96,0.70,355.87,0.74,1.15,0.97,"[1958-02-10, 1978-02-25, 2007-02-05]","[359.79, 366.56, 390.47]",POINT Z (1197817.737 4775715.667 0),-47.049473,167.704481
3,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,30.80,30.80,0.63,0.12,0.64,357.30,0.71,3.90,0.97,0.69,355.92,0.61,0.96,0.98,"[1958-02-10, 1978-02-25, 2007-02-05]","[359.18, 366.97, 389.98]",POINT Z (1197828.167 4775716.541 0),-47.049471,167.704618
4,2.030544e+11,Southland,1958-02-10,2007-02-05,49.0,3.0,31.66,31.66,0.65,0.12,0.66,356.25,0.59,3.27,0.98,0.70,355.09,0.51,0.80,0.99,"[1958-02-10, 1978-02-25, 2007-02-05]","[357.82, 366.74, 389.48]",POINT Z (1197838.278 4775718.205 0),-47.049463,167.704752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228533,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,7.0,-256.06,280.29,-4.97,0.10,-1.70,153.41,3.44,97.44,0.17,-1.98,170.29,3.08,23.83,0.25,"[1969-09-22, 1969-09-22, 1981-01-11, 2010-02-1...","[314.21, 33.91, 68.63, 135.44, 78.3, 51.05, 58...",POINT Z (1616403.774 5429650.065 0),-41.284422,173.195892
228534,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,8.0,-272.22,306.31,-5.29,0.10,-1.58,145.42,3.53,104.92,0.11,-2.28,180.71,3.58,28.75,0.20,"[1969-09-22, 1969-09-22, 1981-01-11, 2006-12-1...","[316.12, 9.81, 46.87, 178.64, 116.03, 52.22, 3...",POINT Z (1616396.401 5429637.534 0),-41.284535,173.195804
228535,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,7.0,-288.24,55.67,-5.60,0.10,-3.75,218.78,3.61,86.99,0.47,-4.64,265.15,3.25,20.37,0.62,"[1969-09-22, 1981-01-11, 2006-12-14, 2010-02-1...","[318.95, 24.48, 135.29, 95.8, 31.5, 16.5, 30.71]",POINT Z (1616388.254 5429628.361 0),-41.284618,173.195707
228536,2.020008e+11,Tasman,1969-09-22,2021-03-18,51.0,8.0,-274.46,302.58,-5.33,0.10,-1.72,148.57,3.38,100.55,0.14,-2.30,178.44,3.25,26.12,0.24,"[1969-09-22, 1969-09-22, 1981-01-11, 2006-12-1...","[319.74, 17.16, 48.95, 156.51, 110.25, 53.65, ...",POINT Z (1616385.177 5429625.739 0),-41.284642,173.195670


In [55]:
rates_with_timeseries.to_excel("rates_with_timeseries.xlsx", index=False)

In [31]:
rates_with_timeseries.UniqueID = rates_with_timeseries.UniqueID.astype(str)
rates_with_timeseries.dtypes

UniqueID        object
Region          object
Start_date      object
End_date        object
Duration       float64
ShrCount       float64
NSM            float64
SCE            float64
EPR            float64
EPRunc         float64
LRR            float64
LRI            float64
LCI            float64
LSE            float64
LR2            float64
WLR            float64
WLI            float64
WCI            float64
WSE            float64
WR2            float64
Dates           object
Distances       object
geometry      geometry
lat            float64
lon            float64
dtype: object

In [53]:
rates_with_timeseries.to_file("rates_with_timeseries.shp")

/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '['1940-04-14', '1949-03-29', '1960-08-19', '1969-04-30', '1980-10-24', '1987-11-17', '2002-04-26', '2009-12-24', '2010-04-07', '2010-11-07', '2011-09-23', '2014-11-29', '2015-03-01', '2016-04-05', '2017-10-23', '2019-03-22', '2020-05-18', '2021-03-21', '2024-04-30']' of field Dates has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(
